In [ ]:
!pip install pandas yfinance tabulate
import pandas as pd
import yfinance as yf
from tabulate import tabulate

In [ ]:
# Load the dataset
df = pd.read_csv('/content/constituents-financials.csv')
print("Dataset loaded. Shape:", df.shape)
df.head(3)

In [ ]:
# Available financial fields
available_fields = df.columns.tolist()
print("Available financial fields:\n", available_fields)

# Example: User wants these
user_requested_entities = ['Name', 'Price', 'Earnings/Share', 'Market Cap', '52 Week High']

# Extract only those columns
def extract_user_entities(dataframe, requested_entities):
    missing = [col for col in requested_entities if col not in dataframe.columns]
    if missing:
        print(f"⚠️ Warning: These fields are not in the dataset: {missing}")
        requested_entities = [col for col in requested_entities if col in dataframe.columns]
    return dataframe[requested_entities]

# Extract
extracted_df = extract_user_entities(df, user_requested_entities)
print("\n✅ Extracted user-defined entities:")
print(tabulate(extracted_df.head(10), headers='keys', tablefmt='pretty'))

In [ ]:
# Create a new column for detected "events"
def detect_financial_events(row):
    events = []
    if row['Price'] >= 0.95 * row['52 Week High']:
        events.append("Near 52-Week High")
    if row['Dividend Yield'] > 5.0:
        events.append("High Dividend (>5%)")
    if row['Earnings/Share'] < 0:
        events.append("Negative EPS (Loss)")
    if row['Price/Earnings'] > 50 and row['Earnings/Share'] > 0:
        events.append("High Valuation (P/E > 50)")
    return "; ".join(events) if events else "None"

# Apply event detection
df['Detected Events'] = df.apply(detect_financial_events, axis=1)

# Show companies with events
event_df = df[df['Detected Events'] != 'None'][['Name', 'Price', 'Earnings/Share', 'Dividend Yield', 'Detected Events']]
print(tabulate(event_df.head(15), headers='keys', tablefmt='pretty'))

In [ ]:
# Create a simple ticker mapping (you can expand this)
ticker_map = {
    'Apple Inc.': 'AAPL',
    'Microsoft Corp.': 'MSFT',
    'Amazon.com Inc': 'AMZN',
    'Alphabet Inc Class A': 'GOOGL',
    'Alphabet Inc Class C': 'GOOG',
    'Tesla Inc': 'TSLA',  # not in your list, but common
    'Johnson & Johnson': 'JNJ',
    'Walmart Inc': 'WMT',  # Note: your dataset says "Wal-Mart Stores"
    'JPMorgan Chase & Co.': 'JPM',
    'Visa Inc.': 'V',
    'Procter & Gamble': 'PG',
    'Coca-Cola Company (The)': 'KO'
}

# Reverse map for lookup
name_to_ticker = {name: ticker for name, ticker in ticker_map.items()}

# Add ticker column
df['Ticker'] = df['Name'].map(name_to_ticker)

# Fetch live price & compare
def enrich_with_yfinance(row):
    if pd.notna(row['Ticker']):
        try:
            ticker = yf.Ticker(row['Ticker'])
            info = ticker.info
            hist = ticker.history(period="1d")
            if not hist.empty:
                live_price = hist['Close'].iloc[-1]
                return round(live_price, 2)
        except Exception as e:
            return row['Price']  # fallback
    return row['Price']

# Only enrich first 10 to avoid rate limits in demo
sample_df = df.head(10).copy()
sample_df['Live Price (Yahoo)'] = sample_df.apply(enrich_with_yfinance, axis=1)

print(tabulate(sample_df[['Name', 'Ticker', 'Price', 'Live Price (Yahoo)']],
               headers='keys', tablefmt='pretty'))

In [ ]:
# Plot Apple's 6-month trend
import matplotlib.pyplot as plt

ticker = yf.Ticker("AAPL")
hist = ticker.history(period="6mo")

plt.figure(figsize=(10, 5))
plt.plot(hist.index, hist['Close'], label='AAPL Close Price')
plt.title("Apple Inc. - 6 Month Stock Price Trend")
plt.xlabel("Date")
plt.ylabel("Price ($)")
plt.legend()
plt.grid(True)
plt.show()

**milestone 3 entity extraction**

In [ ]:
# %% [markdown]
# # 📊 Financial Entity & Event Extraction System (Milestone 3 - Offline-Friendly)
# Rule-based NER + ChromaDB + yfinance
# No LLM dependency — works without API keys!

# %%
# Install required packages (run once)
!pip install pandas yfinance chromadb sentence-transformers tabulate matplotlib ipywidgets

import pandas as pd
import yfinance as yf
import chromadb
from sentence_transformers import SentenceTransformer
from IPython.display import display, clear_output
from ipywidgets import Text, Button, Output, VBox, HBox, Layout, Label
from tabulate import tabulate
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load dataset
df = pd.read_csv('/content/constituents-financials.csv')
print(f"✅ Dataset loaded. Shape: {df.shape}")
print("Columns:", df.columns.tolist())

# Standardize column names if needed (ensure they match expected keys)
expected_cols = ['Name', 'Price', 'Earnings/Share', 'Market Cap', 'Dividend Yield', 'Price/Earnings', '52 Week High', 'Sector']
for col in expected_cols:
    if col not in df.columns:
        print(f"⚠️ Warning: Expected column '{col}' not found.")

# Create ticker map
ticker_map = {
    'Apple Inc.': 'AAPL',
    'Microsoft Corp.': 'MSFT',
    'Amazon.com Inc': 'AMZN',
    'Alphabet Inc Class A': 'GOOGL',
    'Alphabet Inc Class C': 'GOOG',
    'Tesla Inc': 'TSLA',
    'Johnson & Johnson': 'JNJ',
    'Walmart Inc.': 'WMT',
    'JPMorgan Chase & Co.': 'JPM',
    'Visa Inc.': 'V',
    'Procter & Gamble': 'PG',
    'Coca-Cola Company (The)': 'KO',
    'Berkshire Hathaway': 'BRK.B',
    'Home Depot': 'HD',
    'Netflix Inc.': 'NFLX',
    'Meta Platforms, Inc.': 'META',
    'NVIDIA Corporation': 'NVDA',
    'Intel Corp.': 'INTC',
    'Cisco Systems': 'CSCO',
    'Adobe Inc.': 'ADBE',  # Updated name
    'Salesforce Inc.': 'CRM',
    'Pfizer Inc.': 'PFE',
    'Moderna, Inc.': 'MRNA',
    'Merck & Co., Inc.': 'MRK',
    'AbbVie Inc.': 'ABBV',
    'Eli Lilly and Company': 'LLY',
    'UnitedHealth Group Inc.': 'UNH',
    'McDonald\'s Corp.': 'MCD',
    'Starbucks Corp.': 'SBUX',
    'Costco Wholesale Corp.': 'COST',
    'Target Corp.': 'TGT',
    'Walt Disney Company (The)': 'DIS',
    'Boeing Company': 'BA',
    'General Electric': 'GE',
    'Exxon Mobil Corp.': 'XOM',
    'Chevron Corp.': 'CVX',
    'AT&T Inc.': 'T',
    'Verizon Communications Inc.': 'VZ',
    'T-Mobile US, Inc.': 'TMUS',
    'Comcast Corp.': 'CMCSA',
    'Charter Communications, Inc.': 'CHTR',
}

# Create Ticker column — fill missing with empty string
df['Ticker'] = df['Name'].map(ticker_map).fillna('')

# Add event detection
def detect_events(row):
    events = []
    try:
        if pd.notna(row['Price']) and pd.notna(row['52 Week High']):
            if row['Price'] >= 0.95 * row['52 Week High']:
                events.append("Near 52-Week High")
        if pd.notna(row['Dividend Yield']) and row['Dividend Yield'] > 5.0:
            events.append("High Dividend (>5%)")
        if pd.notna(row['Earnings/Share']) and row['Earnings/Share'] < 0:
            events.append("Negative EPS (Loss)")
        if pd.notna(row['Price/Earnings']) and pd.notna(row['Earnings/Share']):
            if row['Price/Earnings'] > 50 and row['Earnings/Share'] > 0:
                events.append("High Valuation (P/E > 50)")
            if row['Price/Earnings'] < 10 and row['Earnings/Share'] > 0:
                events.append("Low P/E (<10)")
    except Exception:
        pass
    return "; ".join(events) if events else "None"

df['Detected Events'] = df.apply(detect_events, axis=1)

# Initialize ChromaDB
chroma_client = chromadb.Client()
collection_name = "financial_companies"
# Delete if exists (for reproducibility in Colab)
try:
    chroma_client.delete_collection(collection_name)
except:
    pass
collection = chroma_client.create_collection(name=collection_name)

# Embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare texts for embedding
df['SearchText'] = df['Name'].fillna('') + " " + df['Sector'].fillna('')
texts = df['SearchText'].tolist()
names = df['Name'].tolist()

# Generate and add embeddings
print("🧠 Generating embeddings...")
embeddings = model.encode(texts, show_progress_bar=False)

collection.add(
    embeddings=embeddings.tolist(),
    documents=texts,
    ids=[str(i) for i in range(len(names))]  # Use numeric IDs to avoid name duplicates
)
# Store name-to-index mapping
df['_chroma_id'] = [str(i) for i in range(len(df))]

# ➤ Rule-based entity extractor (NO LLM NEEDED)
def extract_entities_from_query(user_query):
    q = user_query.lower()
    entities = []
    if "eps" in q or "earnings" in q or "profit" in q:
        entities.append("Earnings/Share")
    if "price" in q or "stock" in q or "share price" in q:
        entities.append("Price")
    if "market cap" in q or "market capitalization" in q:
        entities.append("Market Cap")
    if "dividend" in q:
        entities.append("Dividend Yield")
    if "pe" in q or "p/e" in q or "ratio" in q or "valuation" in q:
        entities.append("Price/Earnings")
    if "52 week" in q or "high" in q:
        entities.append("52 Week High")
    if "sector" in q:
        entities.append("Sector")
    if "event" in q or "alert" in q or "signal" in q:
        entities.append("Detected Events")
    if "ticker" in q or "symbol" in q:
        entities.append("Ticker")
    # Always include Name for display
    return entities if entities else ['Name', 'Price', 'Earnings/Share']

# Yahoo enrichment (safe)
def enrich_with_yahoo(df_subset):
    results = []
    for idx, row in df_subset.head(10).iterrows():
        ticker = row['Ticker']
        use_fallback = True
        enriched = {
            'Live Price (Yahoo)': row.get('Price', 'N/A'),
            'Market Cap (Yahoo)': row.get('Market Cap', 'N/A'),
            'P/E (Yahoo)': row.get('Price/Earnings', 'N/A'),
            'Div Yield % (Yahoo)': row.get('Dividend Yield', 'N/A'),
            'EPS (Yahoo)': row.get('Earnings/Share', 'N/A')
        }
        if ticker and isinstance(ticker, str) and ticker.strip():
            try:
                t = yf.Ticker(ticker)
                hist = t.history(period="1d")
                info = t.info
                if not hist.empty:
                    enriched['Live Price (Yahoo)'] = round(hist['Close'].iloc[-1], 2)
                mc = info.get('marketCap')
                if mc is not None:
                    enriched['Market Cap (Yahoo)'] = mc
                pe = info.get('trailingPE')
                if pe is not None:
                    enriched['P/E (Yahoo)'] = round(pe, 2)
                div_yield = info.get('dividendYield')
                if div_yield is not None:
                    enriched['Div Yield % (Yahoo)'] = round(div_yield * 100, 2)
                eps = info.get('trailingEps')
                if eps is not None:
                    enriched['EPS (Yahoo)'] = round(eps, 2)
                use_fallback = False
            except Exception:
                pass  # silently fall back
        results.append(enriched)
    yahoo_df = pd.DataFrame(results, index=df_subset.head(10).index)
    return pd.concat([df_subset.head(10).reset_index(drop=True), yahoo_df.reset_index(drop=True)], axis=1)

# Output area
output_area = Output()
global_result_df = None

# Widgets
query_input = Text(value='', placeholder='e.g., "Show tech companies near 52-week high"', description='Query:', layout=Layout(width='600px'))
run_button = Button(description="🔍 Run Query", button_style='primary')
download_button = Button(description="📥 Download Results", button_style='success')

def on_run_clicked(b):
    global global_result_df
    with output_area:
        clear_output()
        user_query = query_input.value.strip()
        if not user_query:
            print("⚠️ Please enter a query.")
            return

        print("🔄 Processing...")

        # Step 1: Extract entities from query (rule-based)
        extracted_entities = extract_entities_from_query(user_query)
        print(f"🎯 Extracted Entities: {extracted_entities}")

        # Step 2: Search ChromaDB if query mentions company names
        filtered_df = df.copy()
        if any(kw in user_query.lower() for kw in ['company', 'firm', 'business', 'name', 'show me', 'find']):
            # Perform semantic search
            results = collection.query(query_texts=[user_query], n_results=10)
            matched_ids = [int(id_) for id_ in results['ids'][0]]
            filtered_df = df[df['_chroma_id'].isin([str(i) for i in matched_ids])].copy()

        # Step 3: Apply event filter if requested
        if "event" in user_query.lower() or "alert" in user_query.lower():
            filtered_df = filtered_df[filtered_df['Detected Events'] != "None"]

        # Step 4: Select columns
        cols_to_show = ['Name']
        for col in extracted_entities:
            if col in filtered_df.columns and col != 'Name':
                cols_to_show.append(col)
        # Always add events if relevant
        if 'Detected Events' not in cols_to_show and ("event" in user_query.lower()):
            cols_to_show.append('Detected Events')
        # Ensure Ticker is available for enrichment
        if 'Ticker' not in cols_to_show:
            cols_to_show.append('Ticker')

        cols_to_show = [c for c in cols_to_show if c in filtered_df.columns]
        if filtered_df.empty:
            print("❌ No matching records found.")
            global_result_df = pd.DataFrame()
            return

        filtered_df = filtered_df[cols_to_show].drop_duplicates().reset_index(drop=True)

        # Step 5: Enrich with Yahoo
        result_df = enrich_with_yahoo(filtered_df)

        # Save & display
        global_result_df = result_df.copy()
        print(f"✅ Found {len(result_df)} matching records.\n")
        print(tabulate(result_df.head(15), headers='keys', tablefmt='pretty', showindex=False))

        # Plot if live price exists
        if 'Live Price (Yahoo)' in result_df.columns:
            price_data = result_df[result_df['Live Price (Yahoo)'] != 'N/A']
            if len(price_data) > 0:
                plt.figure(figsize=(10, 4))
                plt.bar(price_data['Name'].head(10), price_data['Live Price (Yahoo)'].head(10), color='steelblue')
                plt.title('Live Stock Prices (Yahoo Finance)')
                plt.ylabel('Price ($)')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.show()

def on_download_clicked(b):
    global global_result_df
    with output_area:
        clear_output()
        if global_result_df is None or global_result_df.empty:
            print("⚠️ No results to download. Please run a query first.")
        else:
            global_result_df.to_csv('financial_extraction_results.csv', index=False)
            print("✅ Saved as 'financial_extraction_results.csv'")
            print("👉 In Colab: Click the 📁 icon on the left to download it.")

run_button.on_click(on_run_clicked)
download_button.on_click(on_download_clicked)

# Build UI
ui = VBox([
    Label("🤖 Ask a natural language question about financial entities or events:"),
    query_input,
    HBox([run_button, download_button]),
    output_area
])

display(ui)